## Node Classification with DGL

>`DGL`은 `GNN`을 편리하게 사용하게 도와주는 패키지 입니다. 그래프계의 `Keras`.! `DGL`은 아래와 같은 장점이 있습니다.

- DGL에서는 `GNN`을 하기 위한 다양한 데이터셋을 제공합니다. 
- `GNN` 모델을 단순히 호출만을 통해 사용할 수 있습니다.
- 모델을 학습할 때 `CPU`, `GPU`를 모두 사용할 수 있습니다.

In [1]:
# 본 튜토리얼에서는 Node Classification에 대해서 다룹니다.

import dgl 
import torch 
import torch.nn as nn 
import torch.nn.functional as F 

c:\Users\EonKim\anaconda3\envs\dgl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading Cora Dataset 

> Cora Dataset은 가장 대표적인 데이터셋 중 하나입니다. Cora Dataset은 논문 간의 인용 정보를 담고 있습니다

In [2]:
import dgl.data 

In [4]:
dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [6]:
# Cora Dataset의 경우 하나의 그래프로 이루어져 있기 때문에 아래와 같은 코드를 사용합니다.
# PPI 등과 같이 Graph Classification을 하는 경우 다양한 그래프가 들어가 있을 수 있습니다. 

g = dataset[0]
print(dataset.__len__())

1


In [7]:
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'feat': Scheme(shape=(1433,), dtype=torch.float32)}
      edata_schemes={'__orig__': Scheme(shape=(), dtype=torch.int64)})

> `dgl.data`는 `ndata`, `edata`등의 정보들을 담고 있습니다.

- `train_mask`: node가 trainset인지 아닌지를 나타냅니다. trainset인 경우 `True`, 아닌 경우에는 `False`로 표시됩니다. 
- `val_mask`: node가 validset인지 아닌지를 나타냅니다.
- `test_mask`: node가 testset인지 아닌지를 나타냅니다.
- `label`: node 카테고리의 ground truth (정답)를 의미합니다.
- `feat`: node의 feature 정보를 의미합니다.
- `ndata`: node의 정보들을 나타냅니다.
- `edata`: edge의 정보들을 나타냅니다.

In [8]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([False, False, False,  ..., False, False, False]), 'label': tensor([4, 4, 4,  ..., 4, 3, 3]), 'val_mask': tensor([False, False,  True,  ..., False, False, False]), 'test_mask': tensor([ True,  True, False,  ..., False, False, False]), 'feat': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]])}
Edge features
{'__orig__': tensor([  298,  9199,  1153,  ..., 10415,  5255,  6356])}


## Defining a Graph Convolutional Network (GCN)

> `GCN`은 Graph Neural Network에서 가장 대표적인 모델 중 하나입니다. 2017년 처음으로 제안되었으며 현재까지 그래프 연구에 기반이 되는 모델입니다. 

In [10]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes): # initialization
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats) # in feature, hidden feature
        self.conv2 = GraphConv(h_feats, num_classes) # hidden feature, output 

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h) # non-linear
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

## Training the GCN

In [11]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # optimizer 
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean() # GCN의 경우 train node에 대해서만 loss를 계산하는 형태로 구성됩니다.
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad() # 초기화
        loss.backward() # back-propagation 
        optimizer.step() # learning rate만큼 step

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

c:\Users\EonKim\anaconda3\envs\dgl\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 1.946, val acc: 0.088 (best 0.088), test acc: 0.085 (best 0.085)
In epoch 5, loss: 1.903, val acc: 0.484 (best 0.484), test acc: 0.488 (best 0.488)
In epoch 10, loss: 1.832, val acc: 0.658 (best 0.658), test acc: 0.667 (best 0.667)
In epoch 15, loss: 1.735, val acc: 0.714 (best 0.714), test acc: 0.707 (best 0.707)
In epoch 20, loss: 1.615, val acc: 0.680 (best 0.714), test acc: 0.672 (best 0.707)
In epoch 25, loss: 1.472, val acc: 0.708 (best 0.714), test acc: 0.683 (best 0.707)
In epoch 30, loss: 1.312, val acc: 0.726 (best 0.726), test acc: 0.725 (best 0.725)
In epoch 35, loss: 1.143, val acc: 0.730 (best 0.730), test acc: 0.741 (best 0.741)
In epoch 40, loss: 0.974, val acc: 0.732 (best 0.732), test acc: 0.746 (best 0.744)
In epoch 45, loss: 0.814, val acc: 0.734 (best 0.736), test acc: 0.754 (best 0.754)
In epoch 50, loss: 0.671, val acc: 0.740 (best 0.740), test acc: 0.755 (best 0.752)
In epoch 55, loss: 0.547, val acc: 0.746 (best 0.746), test acc: 0.758 (best 0

## Training on GPU

> `GPU`를 사용하고 싶은 경우 dataset과 model 둘다 `cuda`로 지정하여야 합니다. 

In [ ]:
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)